# 03_EXPLORE_REPAYMENTS.ipynb
=========================

Objetivo: Explorar y validar el datamart de REPAYMENTS
- ¿Cuántos pagos hay?
- ¿Cuál es el monto promedio de pago?
- ¿Cuándo se realizaron los pagos?
- ¿Qué tipos de eventos hay?
- ¿Cómo se relacionan con los préstamos?

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

In [ ]:
# Cargar datos desde exports
base_path = Path.cwd().parent.parent if 'data_validation' in str(Path.cwd()) else Path.cwd()
data_path = base_path / 'data' / 'exports'

print(f"📁 Working dir: {Path.cwd()}")
print(f"📂 Base path: {base_path}")
print(f"📍 Data path: {data_path}")

# Cargar datos RAW
raw_path = base_path / 'data' / 'raw'

# Cargar repayments RAW
repayments_full = pd.read_csv(raw_path / 'AE_challenge_repayments.csv')

# Cargar loans desde exports (ya modelizado)
loans_full = pd.read_excel(data_path / 'loans.xlsx')
vintage_months_filter = ['2025-01', '2025-02', '2025-03']
loans = loans_full[loans_full['vintage_month'].isin(vintage_months_filter)].copy()

# Filtrar repayments solo por loan_ids que existen en loans de Q1 2025
loan_ids_q1 = set(loans['loan_id'].unique())
repayments = repayments_full[repayments_full['loan_id'].isin(loan_ids_q1)].copy()

print(f"\n✅ Repayments (FILTRADO - Q1 2025): {len(repayments):,} registros de pago")
print(f"   - Total en archivo RAW: {len(repayments_full):,} registros")
print(f"\n✅ Loans (FILTRADO - Q1 2025): {len(loans):,} préstamos")
print(f"   - Total en archivo: {len(loans_full):,} préstamos")
print(f"   - Vintage months: {sorted(loans['vintage_month'].unique())}")

In [ ]:
print("="*80)
print("📊 DATASET: REPAYMENTS")
print("="*80)

# 1. ¿Cuántos registros de pago hay?
print(f"\n1️⃣ Total de registros de pago: {len(repayments):,}")

# 2. ¿Qué columnas tengo?
print(f"\n2️⃣ Columnas disponibles ({len(repayments.columns)}):")
for i, col in enumerate(repayments.columns, 1):
    print(f"   {i:2d}. {col}")

# 3. Ver primeros registros
print(f"\n3️⃣ Primeros 5 registros de pago:")
display(repayments.head())

# 4. Tipos de datos
print(f"\n4️⃣ Tipos de datos:")
print(repayments.dtypes)

# 5. Valores nulos
print(f"\n5️⃣ Valores nulos:")
nulls = repayments.isnull().sum()
nulls_pct = (nulls / len(repayments) * 100).round(2)
null_df = pd.DataFrame({'Nulls': nulls, 'Percentage': nulls_pct})
print(null_df[null_df['Nulls'] > 0])
if null_df[null_df['Nulls'] > 0].empty:
    print("   ✅ Sin valores nulos")

In [ ]:
# 6. Identificar columnas clave
print(f"\n6️⃣ ESTRUCTURA DE DATOS:")

# Encontrar columnas de IDs
id_cols = [col for col in repayments.columns if 'id' in col.lower()]
print(f"   - ID Columns: {id_cols}")

# Encontrar columnas de montos/amounts
amount_cols = [col for col in repayments.columns if any(x in col.lower() for x in ['amount', 'principal', 'revenue', 'interest'])]
print(f"   - Amount Columns: {amount_cols}")

# Encontrar columnas de fecha
date_cols = [col for col in repayments.columns if any(x in col.lower() for x in ['date', 'day', 'month'])]
print(f"   - Date Columns: {date_cols}")

# 7. ¿Hay duplicados?
print(f"\n7️⃣ Verificación de duplicados:")
if 'repayment_id' in repayments.columns:
    duplicates = repayments['repayment_id'].duplicated().sum()
    print(f"   Repayment IDs duplicados: {duplicates}")
elif 'payment_id' in repayments.columns:
    duplicates = repayments['payment_id'].duplicated().sum()
    print(f"   Payment IDs duplicados: {duplicates}")
else:
    print(f"   ⚠️ No hay columna de repayment/payment ID")

# 8. Relación con loans
print(f"\n8️⃣ RELACIÓN CON LOANS:")
loans_in_repayments = repayments['loan_id'].nunique()
coverage = (loans_in_repayments / len(loans) * 100)
print(f"   - Préstamos únicos en repayments: {loans_in_repayments:,}")
print(f"   - Cobertura vs loans table: {coverage:.2f}%")
print(f"   - Pagos promedio por préstamo: {len(repayments) / loans_in_repayments:.2f}")

In [ ]:
# 9. Análisis de MONTOS DE PAGO
amount_col = None
if 'principal_amount' in repayments.columns:
    amount_col = 'principal_amount'
elif 'amount' in repayments.columns:
    amount_col = 'amount'

if amount_col:
    print(f"\n9️⃣ Estadísticas de MONTOS ({amount_col}):")
    print(f"   - Promedio: ${repayments[amount_col].mean():,.2f}")
    print(f"   - Mediana: ${repayments[amount_col].median():,.2f}")
    print(f"   - Mínimo: ${repayments[amount_col].min():,.2f}")
    print(f"   - Máximo: ${repayments[amount_col].max():,.2f}")
    print(f"   - Desv. Est.: ${repayments[amount_col].std():,.2f}")
    print(f"   - TOTAL: ${repayments[amount_col].sum():,.2f}")

# 10. Análisis de FECHAS
date_col = None
if 'event_date' in repayments.columns:
    date_col = 'event_date'
elif 'payment_date' in repayments.columns:
    date_col = 'payment_date'
elif 'date' in repayments.columns:
    date_col = 'date'

if date_col:
    repayments[date_col] = pd.to_datetime(repayments[date_col])
    print(f"\n🔟 Análisis de FECHAS ({date_col}):")
    print(f"   - Desde: {repayments[date_col].min()}")
    print(f"   - Hasta: {repayments[date_col].max()}")
    print(f"\n   Pagos por mes:")
    monthly = repayments.groupby(repayments[date_col].dt.to_period('M')).agg({
        'loan_id': 'count',
        amount_col: 'sum' if amount_col else 'count'
    })
    if amount_col:
        monthly.columns = ['Count', 'Total Amount']
    else:
        monthly.columns = ['Count']
    print(monthly)

In [ ]:
# 11. Tipos de eventos/transacciones
event_cols = [col for col in repayments.columns if any(x in col.lower() for x in ['type', 'event', 'kind', 'category'])]
if event_cols:
    event_col = event_cols[0]
    print(f"\n1️⃣1️⃣ TIPOS DE EVENTOS ({event_col}):")
    print(repayments[event_col].value_counts())
else:
    print(f"\n1️⃣1️⃣ No hay columna de tipo de evento")

# 12. Revenue vs Principal
revenue_cols = [col for col in repayments.columns if 'revenue' in col.lower()]
if revenue_cols and amount_col:
    for rev_col in revenue_cols:
        print(f"\n1️⃣2️⃣ REVENUE ({rev_col}) vs PRINCIPAL ({amount_col}):")
        print(f"   - Total Principal: ${repayments[amount_col].sum():,.2f}")
        print(f"   - Total {rev_col}: ${repayments[rev_col].sum():,.2f}")
        print(f"   - Diferencia (Revenue - Principal): ${(repayments[rev_col].sum() - repayments[amount_col].sum()):,.2f}")
        print(f"   - Ratio Revenue/Principal: {(repayments[rev_col].sum() / repayments[amount_col].sum()):.4f}")

In [ ]:
# 13. Recovery Rate por Préstamo
print(f"\n1️⃣3️⃣ RECOVERY RATE (análisis por préstamo):")

if amount_col and 'loan_id' in repayments.columns:
    # Unir con loans para obtener monto original
    repay_summary = repayments.groupby('loan_id')[amount_col].sum().reset_index()
    repay_summary.columns = ['loan_id', 'total_repaid']
    
    merged = repay_summary.merge(
        loans[['loan_id', 'Total Loan Amount']], 
        on='loan_id', 
        how='left'
    )
    
    merged['recovery_rate'] = merged['total_repaid'] / merged['Total Loan Amount']
    
    print(f"   - Recovery rate promedio: {merged['recovery_rate'].mean():.4f} ({merged['recovery_rate'].mean()*100:.2f}%)")
    print(f"   - Recovery rate mediana: {merged['recovery_rate'].median():.4f} ({merged['recovery_rate'].median()*100:.2f}%)")
    print(f"   - Mínimo: {merged['recovery_rate'].min():.4f}")
    print(f"   - Máximo: {merged['recovery_rate'].max():.4f}")
    
    # Distribución
    print(f"\n   Distribución de recovery rate:")
    print(f"   - 0-25%: {(merged['recovery_rate'] <= 0.25).sum()} préstamos ({(merged['recovery_rate'] <= 0.25).sum() / len(merged) * 100:.2f}%)")
    print(f"   - 25-50%: {((merged['recovery_rate'] > 0.25) & (merged['recovery_rate'] <= 0.50)).sum()} préstamos")
    print(f"   - 50-75%: {((merged['recovery_rate'] > 0.50) & (merged['recovery_rate'] <= 0.75)).sum()} préstamos")
    print(f"   - 75-100%: {((merged['recovery_rate'] > 0.75) & (merged['recovery_rate'] < 1.0)).sum()} préstamos")
    print(f"   - 100%: {(merged['recovery_rate'] >= 1.0).sum()} préstamos ({(merged['recovery_rate'] >= 1.0).sum() / len(merged) * 100:.2f}%)")

In [ ]:
print("\n" + "="*80)
print("✅ CONCLUSIONES - REPAYMENTS")
print("="*80)

if amount_col and date_col:
    total_repaid = repayments[amount_col].sum()
    total_loans_amount = loans['Total Loan Amount'].sum()
    overall_recovery = total_repaid / total_loans_amount
    
    print(f"""
📊 RESUMEN GENERAL:
   Total registros de pago: {len(repayments):,}
   Préstamos con pagos: {loans_in_repayments:,} / {len(loans):,} ({coverage:.2f}%)
   Pagos promedio por préstamo: {len(repayments) / loans_in_repayments:.2f}
   
💰 FINANCIERO:
   Total pagado (principal): ${total_repaid:,.2f}
   Total originado: ${total_loans_amount:,.2f}
   Recovery rate GENERAL: {overall_recovery:.4f} ({overall_recovery*100:.2f}%)
   
📅 PERÍODO DE PAGOS:
   Desde: {repayments[date_col].min()}
   Hasta: {repayments[date_col].max()}
""")
else:
    print("\n⚠️ Falta información para conclusiones completas")